In [1]:
# -*- coding:utf-8 -*-
import tensorflow as tf
from tensorflow.python.ops.image_ops_impl import ResizeMethod

import cifar10
import numpy as np

In [2]:

FLAGS = tf.app.flags.FLAGS
# 设置存储模型训练结果的路径
tf.app.flags.DEFINE_string("checkpoint_dir", "/data/1xiu/models/cifar/cifar_model", "Directory where to read model checkpoints.")
tf.app.flags.DEFINE_string('class_dir', '/tmp/cifar10_data/cifar-10-batches-bin/', """存储文件batches.meta.txt的目录""")
tf.app.flags.DEFINE_string('test_file', 'dog2.jpg', """测试用的图片，可以是绝对路径""")
IMAGE_SIZE = 24

In [3]:
def evaluate_images(images):  # 执行验证
    logits = cifar10.inference(images, batch_size=1)
    predict(logits = logits)


In [40]:
def load_trained_model1():
    with tf.Session() as sess:
        ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
        print (FLAGS.checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            # 从训练模型恢复数据
            saver = tf.train.Saver()
           
            saver.restore(sess, ckpt.model_checkpoint_path)
        else:
            print('No checkpoint file found')
            return

        # 下面两行是预测最有可能的分类
        # predict = tf.argmax(logits, 1)
        # output = predict.eval()

        # 从文件以字符串方式获取10个类标签，使用制表格分割
        cifar10_class = np.loadtxt(FLAGS.class_dir + "batches.meta.txt", str, delimiter='\t')



IndentationError: unindent does not match any outer indentation level (<ipython-input-40-d29aad3ae903>, line 19)

In [45]:
def load_trained_model():
    with tf.Session() as sess:
       
        # 从训练模型恢复数据
        saver = tf.train.Saver()
        saver.restore(sess, "/data/1xiu/models/cifar/cifar_model")
        
        # 下面两行是预测最有可能的分类
        # predict = tf.argmax(logits, 1)
        # output = predict.eval()

        # 从文件以字符串方式获取10个类标签，使用制表格分割
        cifar10_class = np.loadtxt(FLAGS.class_dir + "batches.meta.txt", str, delimiter='\t')

In [46]:
def predict(logits):
    with tf.Session() as sess:
            # 预测最大的三个分类
        top_k_pred = tf.nn.top_k(logits, k=3)
        output = sess.run(top_k_pred)
        probability = np.array(output[0]).flatten()  # 取出概率值，将其展成一维数组
        index = np.array(output[1]).flatten()
        # 使用表格的方式显示
        tabel = PrettyTable(["index", "class", "probability"])
        tabel.align["index"] = "l"  # 左边界
        tabel.padding_width = 1  # padding值，默认为1
        for i in np.arange(index.size):
            tabel.add_row([index[i], cifar10_class[index[i]], probability[i]])
        print tabel

In [47]:
def img_read(filename):
    if not tf.gfile.Exists(filename):
        tf.logging.fatal('File does not exists %s', filename)
    image_data = tf.image.convert_image_dtype(tf.image.decode_jpeg(tf.read_file(filename),
                                                                   channels=3), dtype=tf.float32)
    height = IMAGE_SIZE
    width = IMAGE_SIZE
    image = tf.image.resize_images(image_data, (height, width), method=ResizeMethod.BILINEAR)
    image = tf.expand_dims(image, -1)
    image = tf.reshape(image, (1, 24, 24, 3))
    return image

In [52]:
def main(argv=None):  # pylint: disable=unused-argument
    #load_trained_model()
    saver = tf.train.Saver()
    model_dir = "/data/1xiu/models//cifar"
    model_name = "cifar_model"
 
    saver.restore(sess, os.path.join(model_dir, model_name))
    filename = FLAGS.test_file
    print(filename)
    images = img_read(filename)
    evaluate_images(images)


if __name__ == '__main__':
    tf.app.run()

ValueError: No variables to save